In [167]:
import cv2
import numpy as np

In [168]:
img = cv2.imread('img1.jpg')
temp_img = img.copy()

click = False
x1,y1 = -1,-1
x2,y2 = -1,-1
alpha = 0

COLOR_WHITE = (255,255,255)
COLOR_BLUE = (255,0,0)
COLOR_GREEN = (0,255,0)
COLOR_RED = (0,0,255)

In [169]:
def coordinate_normalized():
    global x1,y1,x2,y2
    c_temp1, c_temp2 = x1, y1
    
    x1 = min(c_temp1, x2)
    y1 = min(c_temp2, y2)
    x2 = max(c_temp1, x2)
    y2 = max(c_temp2, y2)

In [170]:
def draw_alpha():
    coordinate_normalized()
    
    for i in range(0, abs(x2-x1)):
        for j in range(0, abs(y2-y1)):
            img.itemset((y1+j,x1+i, 2), 255-alpha)

In [171]:
def draw_inner_rectangle():
    coordinate_normalized()
    temp_f_img = temp_img[y1:y2, x1:x2]
    #cv2.imshow("temp", temp_f_img)

    (B, G, R) = cv2.split(temp_f_img)
    #cv2.imshow("Red", R)

    zeros = np.zeros(temp_f_img.shape[:2], dtype = "uint8")
    red_img = cv2.merge([zeros, zeros, R])
    #cv2.imshow("Red", red_img)

    dst = cv2.addWeighted(red_img,((100-alpha)/100), temp_f_img,(alpha/100), 0)
    #cv2.imshow("add", dst)

    img[y1:y2, x1:x2] = dst

In [172]:
def nothing(x):
    #draw_alpha()
    draw_inner_rectangle()

In [173]:
def draw_rectangle(event, x, y, flags, param):
    global x1,y1,x2,y2, click, img

    if event == cv2.EVENT_LBUTTONDOWN:
        if not(click):
            img = temp_img.copy()
        click = True
        x1, y1 = x,y
		
    elif event == cv2.EVENT_MOUSEMOVE:
        if click == True:
            img = temp_img.copy()
            cv2.rectangle(img,(x1,y1),(x,y),COLOR_GREEN,2)

    elif event == cv2.EVENT_LBUTTONUP:
        click = False
        x2, y2 = x,y
        cv2.rectangle(img,(x1,y1),(x,y),COLOR_GREEN,1)
        #draw_alpha()
        draw_inner_rectangle()
        cv2.putText(img, "Mouse position ("+str(x1)+","+str(y1)+") - ("+str(x2)+","+str(y2)+")", (10,25), cv2.FONT_ITALIC, 0.5, COLOR_WHITE, 1)

In [174]:
cv2.namedWindow('image')
cv2.createTrackbar('value','image',0,100,nothing)
cv2.setMouseCallback('image',draw_rectangle)

while(1):
    cv2.imshow('image',img)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

    alpha = cv2.getTrackbarPos('value','image')
    
cv2.destroyAllWindows()
